In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()
# Is this eager mode ?
tf.executing_eagerly()
assert(tf.__version__ == '2.0.0')

import os
from os import listdir
from os.path import join
import pathlib
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

In [3]:
DATASET_ROOT = '/home/cszsolnai/datasets/MICCAI_BraTS_2018_Data_Training'
HGG = join(DATASET_ROOT, 'HGG')
LGG = join(DATASET_ROOT, 'LGG')

# Slice can be from 0 to 154
SLICE = 80 

In [4]:
root = pathlib.Path(HGG)

In [11]:
t1ce = sorted([str(e) for e in root.glob("*/*t1ce.nii.gz")])
seg = sorted([str(e) for e in root.glob("*/*seg.nii.gz")])

In [21]:
t1ce_ds = tf.data.Dataset.list_files(str(root/'*/*t1ce.nii.gz'), shuffle=False)
seg_ds = tf.data.Dataset.list_files(str(root/'*/*seg.nii.gz'), shuffle=False)
#t1ce_ds = tf.data.TextLineDataset(t1ce)
#seg_ds = tf.data.TextLineDataset(seg)

In [22]:
def get_brain_slice(path):
    #return tf.io.read_file(path)
    return nib.load(path.numpy()).get_data()[:, :, SLICE]

In [39]:
def preprocess_batch(batch):
    # No actual preprocessing, just open the file
    b = []
    for e in batch:
        b.append(nib.load(e.numpy().decode()).get_data()[:, :, SLICE])
    return b

In [40]:
#labeled_ds = tf.data.Dataset.zip((t1ce_ds.map(get_brain_slice), seg_ds.map(get_brain_slice)))
labeled_ds = tf.data.Dataset.zip((t1ce_ds, seg_ds))

In [41]:
dataset = labeled_ds.shuffle(buffer_size=100).batch(5)

In [42]:
l1 = []
l2 = []
for data, label in dataset.take(1):
    l1.append(preprocess_batch(data))
    l2.append(preprocess_batch(label))